Librerias

In [111]:
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
import re

Funciones

In [130]:

###
## Funcion que lee de un proyecto los todos los issues abiertos y cerrados.
## con fines, educaciones y te prueba de concepto.
###
def LeerurlIssues(url,repo,proy,tipo):
    #Rutas Fijas para identificar Issues Abiertos y Cerrados
    urlabiertos = '/issues?q=is%3Aopen+is%3Aissue'
    urlcerrados = '/issues?q=is%3Aissue+is%3Aclosed'
    patronbusqueda = repo+proy+'/'+tipo+'/'

    
    soupopen = leerUrlParametros(url,repo,proy,urlabiertos)
    linksopen = soupopen('a')
    
    dfabiertos = ExtraerLinkIssues(url,linksopen,patronbusqueda)
    soupclosed = leerUrlParametros(url,repo,proy,urlcerrados)
    linksclosed = soupclosed('a')
    dfcerrados = ExtraerLinkIssues(url,linksclosed,patronbusqueda)
    
    df = pd.concat([dfabiertos, dfcerrados], axis=0)
    return df  

def ExtraerLinkIssues(url,linkPage,patron):
    lista = []
    for tag in linkPage:
        valor = tag.get('href')
        if(valor.find(patron) != -1):
            
            lista.append(url+ valor)        
    df = pd.DataFrame (lista, columns = ['url_issues'])
    df = df.drop_duplicates()
    return df

def leerUrlParametros(url,repo,proy,tipo):      
    pagina =   url+repo+proy+tipo
    soup = BeautifulSoup(urllib.request.urlopen(pagina).read().decode())
    return  soup

def leerUrl(pagina):    
    soup = BeautifulSoup(urllib.request.urlopen(pagina).read().decode())
    return  soup

def IssuesToDataFrame(lista):
    columns = ["issue", "author", "state", "assignees"]
    df = pd.DataFrame(columns=columns)

    for i in range(len(lista)):
        link = str(lista.iloc[i]['url_issues'])        
        d = ParsearPagina(link)
        df = pd.concat([df, d], axis=0)
    return df


def ParsearPagina (link):
    page = leerUrl(link)
    # Con el DataFrame de las paginas de Issues se procede a convertirlo en el DataFrame tarea
    # definición del DataFrame
    columns = ["issue", "author", "state", "assignees"]
    df = pd.DataFrame(columns=columns)
    state = "open"
    page.find_all(content= re.compile('https://github.com/pescap/WebScrapingMDS/issues/61' +"$"))
    if page.find("span", attrs={"title": "Status: Closed"}):
        state = "closed"
    state
    author = page.find("a", attrs={"class": "author text-bold Link--secondary"}).get_text()
    author
    assignees = (
            page.find("span", attrs={"class": "css-truncate js-issue-assignees"})
            .get_text()
            .replace("\n", "")
            .split()
        )

    if assignees == ["No", "one", "assigned"]:
        assignees = ["No one assigned"]
    assignees
    df = df.append(
                {"issue": '8', "author": author, "state": state, "assignees": assignees},
                ignore_index=True,
            )
    return df

def ExtraerLinkPaginasOfertas(linkPage,patronBusqueda):
    listaofertas = []
    for tag in linkPage:
        valor = tag.get('href')
        if(valor.find(patronBusqueda) != -1):
            listaofertas.append('https://postulaciones.trabajaenitau.cl'+valor)
    return listaofertas


In [97]:
PaginaRepositorio = '/pescap'
UrlBase = 'https://github.com'
Proyecto = '/WebScrapingMDS'
TipoGitHub = 'issues'

#page = leerUrlParametros(UrlBase,PaginaRepositorio,Proyecto,TipoGitHub)

pr = LeerurlIssues(UrlBase,PaginaRepositorio,Proyecto,TipoGitHub)
display(pr)


,url_issues
0,https://github.com/pescap/WebScrapingMDS/issue...
2,https://github.com/pescap/WebScrapingMDS/issue...
4,https://github.com/pescap/WebScrapingMDS/issue...
6,https://github.com/pescap/WebScrapingMDS/issue...
9,https://github.com/pescap/WebScrapingMDS/issue...
11,https://github.com/pescap/WebScrapingMDS/issue...
14,https://github.com/pescap/WebScrapingMDS/issue...
17,https://github.com/pescap/WebScrapingMDS/issue...
20,https://github.com/pescap/WebScrapingMDS/issue...
23,https://github.com/pescap/WebScrapingMDS/issue...


In [131]:
df_Final = IssuesToDataFrame(pr)
df_Final

C:\Users\jorge\AppData\Local\Temp\ipykernel_16268\2306374121.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\jorge\AppData\Local\Temp\ipykernel_16268\2306374121.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\jorge\AppData\Local\Temp\ipykernel_16268\2306374121.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\jorge\AppData\Local\Temp\ipykernel_16268\2306374121.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\jorge\AppData\Local\Temp\ipykernel_16268\2306374121.py:77: FutureWarning: The frame.append method is deprecated and

,issue,author,state,assignees
0,8,jwilenmann,open,[No one assigned]
0,8,gavalenz,open,[No one assigned]
0,8,jwilenmann,open,[No one assigned]
0,8,GeorgeAdrock,open,[No one assigned]
0,8,MiguelAbarzua,open,[No one assigned]
0,8,ronaldchester,open,[No one assigned]
0,8,Carloslugook,open,[No one assigned]
0,8,omaldonados,open,[No one assigned]
0,8,pescap,closed,[No one assigned]
0,8,DavidValenzuelaP,open,[No one assigned]
